In [70]:
from google.cloud import bigquery
import pandas as pd

In [71]:

time = '2025-06-15'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP({time})
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df = results.to_dataframe()
df.head(2)

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time
0,8010962861,"[{'key': 'total_time', 'value': '9.79925131797...",אני מבינה שאהבתם את <strong>The Bold Type</str...,response answer,Desktop,2025-06-24 04:40:26.372000+00:00
1,8010962861,"[{'key': 'total_time', 'value': '9.79925131797...",אני מבינה שאהבתם את <strong>The Bold Type</str...,response answer,Desktop,2025-06-24 04:40:22.930000+00:00


In [ ]:
[{'key': 'total_time', 'value': '9.799251317977905'}
 {'key': 'rag_speed', 'value': '7.32828164100647'}
 {'key': 'session_id', 'value': '110'}
 {'key': 'timestamp', 'value': '1.7507400225755832E9'}
 {'key': 'sso_id', 'value': '8010962861'}
 {'key': 'input_tokens', 'value': '20'}
 {'key': 'llm_speed', 'value': '2.4678661823272705'}
 {'key': 'thinking_process', 'value': 'false'}
 {'key': 'query', 'value': 'The Bold Type'}
 {'key': 'article_ids_2', 'value': '00000191-b64b-dd1b-a7db-b7df47a90000'}
 {'key': 'output_tokens', 'value': '212'}
 {'key': 'writer_filter', 'value': ''} {'key': 'version', 'value': '1.1'}
 {'key': 'article_ids_1', 'value': '00000195-177c-d57f-a9df-77fd0a8e0000'}
 {'key': 'error', 'value': '0'}
 {'key': 'remaining_user_messages', 'value': '199'}
 {'key': 'troll_triggered', 'value': 'false'}
 {'key': 'conversation_key', 'value': '8010962861_110'}
 {'key': 'genres', 'value': ''} {'key': 'regenerate', 'value': 'false'}
 {'key': 'media_type', 'value': 'series'}
 {'key': 'streaming_platforms', 'value': ''}]

In [72]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
expanded_df = pd.json_normalize(df['parsed_data'].tolist())
expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,total_time,rag_speed,session_id,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,8010962861,אני מבינה שאהבתם את <strong>The Bold Type</str...,response answer,Desktop,2025-06-24 04:40:26.372000+00:00,9.799251317977905,7.32828164100647,110,1.7507400225755832E9,20,...,00000195-177c-d57f-a9df-77fd0a8e0000,0,199,false,8010962861_110,,false,series,,8010962861_110
1,8010962861,אני מבינה שאהבתם את <strong>The Bold Type</str...,response answer,Desktop,2025-06-24 04:40:22.930000+00:00,9.799251317977905,7.32828164100647,110,1.7507400225755832E9,20,...,00000195-177c-d57f-a9df-77fd0a8e0000,0,199,false,8010962861_110,,false,series,,8010962861_110


id
3057469657_43     12
8010962861_109     5
8010962861_110     3
dtype: int64

In [76]:
df[df['id'] == '3057469657_43']

,sso_id,campaign_details,campaign_name,platform,event_time,total_time,rag_speed,session_id,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
8,3057469657,None,response answer,Mobile,2025-06-24 03:43:37.605000+00:00,3.347423553466797,0.302060604095459,43,1.7507365875535667E9,71828,...,00000188-de74-df52-a79d-de770e890000,NaN,195,false,3057469657_43,"קומדיה, רומנטיקה",false,,,3057469657_43
9,3057469657,"בטח, הנה שתי המלצות בשבילכם:\n\n<strong>פבלו א...",response answer,Mobile,2025-06-24 03:43:26.764000+00:00,3.347423553466797,0.302060604095459,43,1.7507365875535667E9,71828,...,00000188-de74-df52-a79d-de770e890000,NaN,195,false,3057469657_43,"קומדיה, רומנטיקה",false,,,3057469657_43
10,3057469657,"בטח, הנה שתי המלצות בשבילכם:\n\n<strong>פבלו א...",response answer,Mobile,2025-06-24 03:43:17.237000+00:00,3.347423553466797,0.302060604095459,43,1.7507365875535667E9,71828,...,00000188-de74-df52-a79d-de770e890000,NaN,195,false,3057469657_43,"קומדיה, רומנטיקה",false,,,3057469657_43
11,3057469657,קומדיה רומנטיצ,open question,Mobile,2025-06-24 03:43:03.870000+00:00,NaN,NaN,43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3057469657_43
12,3057469657,נשמע טוב,open question,Mobile,2025-06-24 03:42:52.995000+00:00,NaN,NaN,43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3057469657_43
13,3057469657,"בטח, יש לי המלצה מצוינת עבורכם מ<strong>חן חדד...",response answer,Mobile,2025-06-24 03:41:53.667000+00:00,5.058245897293091,0.29380106925964355,43,1.7507365054183033E9,35112,...,00000196-d2f1-db37-addf-f6f511250000,NaN,197,false,3057469657_43,,false,,,3057469657_43
14,3057469657,"בטח, יש לי המלצה מצוינת עבורכם מ<strong>חן חדד...",response answer,Mobile,2025-06-24 03:41:48.362000+00:00,5.058245897293091,0.29380106925964355,43,1.7507365054183033E9,35112,...,00000196-d2f1-db37-addf-f6f511250000,NaN,197,false,3057469657_43,,false,,,3057469657_43
15,3057469657,כן,open question,Mobile,2025-06-24 03:41:40.126000+00:00,NaN,NaN,43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3057469657_43
16,3057469657,נראה משעמם,open question,Mobile,2025-06-24 03:41:29.978000+00:00,NaN,NaN,43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3057469657_43
17,3057469657,"בטח, הנה המלצה מ<strong>שני ליטמן</strong>:\n\...",response answer,Mobile,2025-06-24 03:40:06.906000+00:00,3.774388074874878,0.8415017127990723,43,1.7507364015558703E9,9,...,00000194-4be7-dff1-a7bc-dbf7c5430000,NaN,199,false,3057469657_43,,false,,,3057469657_43


In [93]:
def parse_conversation(ids:str) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            text = f"{row['event_time']} - USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            text = f"{row['event_time']} - LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

print(parse_conversation('3057469657_43'))

2025-06-24 03:39:57.258000+00:00 - USER: תמליץ על משהו 
2025-06-24 03:40:05.533000+00:00 - LLM: בטח, הנה המלצה מ<strong>שני ליטמן</strong>:

הסרט <strong>\"אמון\"</strong> בבימויו של דניאלה לוקטי, מבוסס על הספר \"שבועת אמונים\" מאת דומניקו סטרנונה. זהו סרט דרמטי על מורה ותלמידה לשעבר שאהבתם הופכת לברית גורלית סביב סודותיהם הכמוסים. הסרט מתאר את חייהם לאורך כארבעה עשורים, ומטריד שאלות על הפער בין פנימיותו של אדם לתדמיתו החיצונית. <strong>שני ליטמן</strong> מציינת שהסרט מתעתע בציפיות הצופים ומגלה שהחידה נוגעת לקשרים בין בני אדם בכלל: מה חזק יותר, הקשרים שנוצרים מאהבה או מפחד? בניגוד לסרטים איטלקיים אחרים, הסרט לא מתרחש בנופים עוצרי נשימה אלא בפרברים אפורים של רומא, מה שמדגיש את הצד האנושי והפשוט של הסיפור.


2025-06-24 03:40:06.906000+00:00 - LLM: בטח, הנה המלצה מ<strong>שני ליטמן</strong>:

הסרט <strong>\"אמון\"</strong> בבימויו של דניאלה לוקטי, מבוסס על הספר \"שבועת אמונים\" מאת דומניקו סטרנונה. זהו סרט דרמטי על מורה ותלמידה לשעבר שאהבתם הופכת לברית גורלית סביב סודותיהם הכמוסים. הסרט מ

In [82]:
df[df['id'] == '8010962861_110']

,sso_id,campaign_details,campaign_name,platform,event_time,total_time,rag_speed,session_id,timestamp,input_tokens,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,8010962861,אני מבינה שאהבתם את <strong>The Bold Type</str...,response answer,Desktop,2025-06-24 04:40:26.372000+00:00,9.799251317977905,7.32828164100647,110,1.7507400225755832E9,20,...,00000195-177c-d57f-a9df-77fd0a8e0000,0,199,false,8010962861_110,,false,series,,8010962861_110
1,8010962861,אני מבינה שאהבתם את <strong>The Bold Type</str...,response answer,Desktop,2025-06-24 04:40:22.930000+00:00,9.799251317977905,7.32828164100647,110,1.7507400225755832E9,20,...,00000195-177c-d57f-a9df-77fd0a8e0000,0,199,false,8010962861_110,,false,series,,8010962861_110
2,8010962861,אהבתי את the bold type. יש לך עוד סדרות בסגנון?,open question,Desktop,2025-06-24 04:40:10.417000+00:00,NaN,NaN,110,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8010962861_110


In [87]:
SQL_QUERY = """
SELECT *
FROM `htz-common.raw_data.requests`,
UNNEST(additional_info) as info
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP("2025-06-24")
  AND page_type = 'Chatbot'
  AND sso_id = '8010962861'
  AND info.key = 'session_id'
  AND info.value = '110'
"""

query_job = client.query(SQL_QUERY)
results = query_job.result()
temp = results.to_dataframe()
temp
#

BadRequest: 400 No matching signature for operator = for argument types: INT64, STRING
  Signature: T1 = T1
    Unable to find common supertype for templated argument <T1>
      Input types for <T1>: {INT64, STRING} at [11:9]; reason: invalidQuery, location: query, message: No matching signature for operator = for argument types: INT64, STRING
  Signature: T1 = T1
    Unable to find common supertype for templated argument <T1>
      Input types for <T1>: {INT64, STRING} at [11:9]

Location: EU
Job ID: 6e4ae6a6-8723-4261-8233-732cd1871664


In [ ]:
df[df['campaign_details'].fillna('').str.contains('אמון')]

17    3057469657_43
18    3057469657_43
Name: id, dtype: object

In [ ]:
בטח, הנה המלצה מ<strong>שני ליטמן</strong>:

הסרט <strong>\"אמון\"</strong> בבימויו של דניאלה לוקטי, מבוסס על הספר \"שבועת אמונים\" מאת דומניקו סטרנונה. זהו סרט דרמטי על מורה ותלמידה לשעבר שאהבתם הופכת לברית גורלית סביב סודותיהם הכמוסים. הסרט מתאר את חייהם לאורך כארבעה עשורים, ומטריד שאלות על הפער בין פנימיותו של אדם לתדמיתו החיצונית. <strong>שני ליטמן</strong> מציינת שהסרט מתעתע בציפיות הצופים ומגלה שהחידה נוגעת לקשרים בין בני אדם בכלל: מה חזק יותר, הקשרים שנוצרים מאהבה או מפחד? בניגוד לסרטים איטלקיים אחרים, הסרט לא מתרחש בנופים עוצרי נשימה אלא בפרברים אפורים של רומא, מה שמדגיש את הצד האנושי והפשוט של הסיפור.

